In [1]:
import subprocess
import tempfile
import os
import re

def run_mypy(code, install_stubs=False):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".py", mode="w") as temp_file:
        temp_file_name = temp_file.name
        temp_file.write(code)

    # First run of mypy to check for missing stubs
    first_run = subprocess.run(["mypy", temp_file_name], capture_output=True, text=True)
    if install_stubs and ("install-types" in first_run.stderr or "install-types" in first_run.stdout):
        print("Installing missing stubs...")
        # Install missing stubs
        subprocess.run(["mypy", "--install-types", "--non-interactive"], capture_output=True, text=True)
        # Re-run mypy after installing stubs
        second_run = subprocess.run(["mypy", temp_file_name], capture_output=True, text=True)
        result = second_run.stdout if second_run.stdout else second_run.stderr
    else:
        print("No missing stubs found.")
        result = first_run.stdout if first_run.stdout else first_run.stderr

    os.remove(temp_file_name)
    return result

In [2]:
code = """
import numpy as np
from pandas import DataFrame

from llmcoder.utils import get_data_dir

print(np.array([1,2,3]))
df = DataFrame({'a': [1,2,3], 'b': [4,5,6]})
illegal_df = DataFrame(False)

print(get_data_dir(1))
"""

print(run_mypy(code, install_stubs=True))

No missing stubs found.
/tmp/tmpuoztgpml.py:9: error: No overload variant of "DataFrame" matches argument type "bool"  [call-overload]
/tmp/tmpuoztgpml.py:9: note: Possible overload variants:
/tmp/tmpuoztgpml.py:9: note:     def DataFrame(data: Sequence[Any] | ndarray[Any, Any] | Series[Any] | Index[Any] | DataFrame | dict[Any, Any] | Iterable[Sequence[Any] | ndarray[Any, Any] | Series[Any] | Index[Any] | tuple[Hashable, Sequence[Any] | ndarray[Any, Any] | Series[Any] | Index[Any]] | dict[Any, Any]] | None = ..., index: Index[Any] | Series[Any] | ndarray[Any, Any] | list[Any] | dict[Any, Any] | range | tuple[Any, ...] | None = ..., columns: Index[Any] | Series[Any] | ndarray[Any, Any] | list[Any] | dict[Any, Any] | range | tuple[Any, ...] | None = ..., dtype: Any = ..., copy: bool = ...) -> DataFrame
/tmp/tmpuoztgpml.py:9: note:     def DataFrame(data: str | bytes | date | datetime | timedelta | datetime64 | timedelta64 | bool | int | float | Timestamp | Timedelta | complex, index: Ind